In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
unp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Unpaid_Installments_sample.csv")
unp.printSchema()
unp.registerTempTable('u')
unp.show(2)

root
 |-- Keydate_prompt: integer (nullable = true)
 |-- OIB: long (nullable = true)
 |-- CONTRACT_ACCOUNT: long (nullable = true)
 |-- Installment_Plan_No: integer (nullable = true)
 |-- Due_Date_YYYYMMDD: integer (nullable = true)
 |-- Due_Days_Inst_on_KeyDate: integer (nullable = true)
 |-- Amount: double (nullable = true)

+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+
|Keydate_prompt|        OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|
+--------------+-----------+----------------+-------------------+-----------------+------------------------+------+
|      20170731|49304755245|      4000087477|          210591030|         20170717|                      14| 651.0|
|      20170731|49304755245|      4000087477|          210591030|         20170617|                      44|651.87|
+--------------+-----------+----------------+-------------------+-----------------+--------

In [3]:
lock=spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Lock_by_DocNum_-_Complaints.csv")
lock.printSchema()
lock.registerTempTable('l')

root
 |-- Document_Number: long (nullable = true)
 |-- Lock_From_Date: integer (nullable = true)
 |-- Lock_To_Date: integer (nullable = true)



In [7]:
####Change is here only rest is similar to case 7####
df=spark.sql("select * from u inner join l on u.Installment_Plan_No=l.Document_Number \
WHERE u.Keydate_prompt between l.Lock_From_Date AND l.Lock_To_Date")
df.show()
print(df)

+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+
|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Document_Number|Lock_From_Date|Lock_To_Date|
+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+
+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+

DataFrame[Keydate_prompt: int, OIB: bigint, CONTRACT_ACCOUNT: bigint, Installment_Plan_No: int, Due_Date_YYYYMMDD: int, Due_Days_Inst_on_KeyDate: int, Amount: double, Document_Number: bigint, Lock_From_Date: int, Lock_To_Date: int]


In [8]:
df1=df.withColumn("Interval",f.expr("case when Due_Days_Inst_on_KeyDate<=0 then 'ON_TIME'\
                                        when Due_Days_Inst_on_KeyDate>=44 and Due_Days_Inst_on_KeyDate<=63 then 4463\
                                        when Due_Days_Inst_on_KeyDate>=64 and Due_Days_Inst_on_KeyDate<=93 then 6493\
                                        when Due_Days_Inst_on_KeyDate>=94  then 94\
                                        when Due_Days_Inst_on_KeyDate>=1 and Due_Days_Inst_on_KeyDate<=43 then 'remainder' end"))

df1.show()
df1.registerTempTable('v')

+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+--------+
|Keydate_prompt|OIB|CONTRACT_ACCOUNT|Installment_Plan_No|Due_Date_YYYYMMDD|Due_Days_Inst_on_KeyDate|Amount|Document_Number|Lock_From_Date|Lock_To_Date|Interval|
+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+--------+
+--------------+---+----------------+-------------------+-----------------+------------------------+------+---------------+--------------+------------+--------+



In [9]:
df2=spark.sql("select v.Keydate_prompt,v.oib,v.amount,v.Interval from v \
group by v.Keydate_prompt,v.oib,v.amount,v.Interval")
                                    
df2.show(2)

+--------------+---+------+--------+
|Keydate_prompt|oib|amount|Interval|
+--------------+---+------+--------+
+--------------+---+------+--------+

